# Check GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Dec  3 04:37:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Run this command once and restart the runtime

In [ ]:
!pip install av

     |████████████████████████████████| 36.9MB 132kB/s 


# HMDB51 + DataLoader

In [ ]:
import glob
import os

from torchvision.datasets.utils import list_dir
from torchvision.datasets.folder import make_dataset
from torchvision.datasets.video_utils import VideoClips
from torchvision.datasets.vision import VisionDataset

class HMDB51(VisionDataset):
    """
    Internally, it uses a VideoClips object to handle clip creation.
    Args:
        root (string): Root directory of the HMDB51 Dataset.
        frames_per_clip (int): Number of frames in a clip.
        step_between_clips (int): Number of frames between each clip.
        train (bool, optional): If ``True``, creates a dataset from the train split,
            otherwise from the ``test`` split.
        transform (callable, optional): A function/transform that takes in a TxHxWxC video
            and returns a transformed version.
    Returns:
        video (Tensor[T, H, W, C]): the `T` video frames
        audio(Tensor[K, L]): the audio frames, where `K` is the number of channels
            and `L` is the number of points
        label (int): class of the video clip
    """
    def __init__(self, root=""):
        super(HMDB51, self).__init__(root)

    def init_data(self, root, frames_per_clip, step_between_clips=1,
                 frame_rate=None, train=True, transform=None,
                 _precomputed_metadata=None, num_workers=1, _video_width=0,
                 _video_height=0, _video_min_dimension=0, _audio_samples=0):
        super(HMDB51, self).__init__(root)
        extensions = ('avi',)
        if train:
            root = root + "/train"
        else:
            root = root + "/test"
        classes = sorted(list_dir(root))
        class_to_idx = {class_: i for (i, class_) in enumerate(classes)}
        print(class_to_idx)
        self.samples = []
        for target_class in sorted(class_to_idx.keys()):
            class_index = class_to_idx[target_class]
            target_dir = os.path.join(root, target_class)
            for root_curr, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                for fname in sorted(fnames):
                    path = os.path.join(root_curr, fname)
                    if os.path.isfile(path):
                        item = path, class_index
                        self.samples.append(item)

        video_paths = [path for (path, _) in self.samples]
        video_clips = VideoClips(
            video_paths,
            frames_per_clip,
            step_between_clips,
            frame_rate,
            _precomputed_metadata,
            num_workers=num_workers,
            _video_width=_video_width,
            _video_height=_video_height,
            _video_min_dimension=_video_min_dimension,
            _audio_samples=_audio_samples,
        )
        self.train = train
        self.classes = classes
        self.video_clips_metadata = video_clips.metadata
        self.indices = self.get_indices(video_paths)
        self.video_clips = video_clips.subset(self.indices)
        self.transform = transform

    @property
    def metadata(self):
        return self.video_clips_metadata

    def get_indices(self, video_list):
        indices = []
        for video_index, video_path in enumerate(video_list):
            indices.append(video_index)
        return indices

    def __len__(self):
        return self.video_clips.num_clips()

    def __getitem__(self, idx):
        video, _, _, video_idx = self.video_clips.get_clip(idx)
        sample_index = self.indices[video_idx]
        _, class_index = self.samples[sample_index]

        if self.transform is not None:
            video = video.permute(0, 3, 1, 2)
            video = self.transform(video)
            video = video.squeeze(0)

        return video, class_index, sample_index

    def state_dict(self):
        state = {"video_clips": self.video_clips,
                 "indices": self.indices,
                 "samples": self.samples,
                 "transform": self.transform,
                 "metadata": self.video_clips_metadata}

        return state

    def load_state_dict(self, state):
        self.video_clips = state["video_clips"]
        self.indices = state["indices"]
        self.samples = state["samples"]
        self.transform = state["transform"]
        self.video_clips_metadata = state["metadata"]

import torch
from torch.utils.data import DataLoader

class BuildDataLoader(torch.utils.data.DataLoader):
    def __init__(self, dataset, batch_size, shuffle, num_workers):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_workers = num_workers
        self.w_frame = 224
        self.h_frame = 224

    def collect_fn(self, batch):
        video_list = []
        label_list = []
        index_list = []

        for video, cl_index, s_index in batch:
            video_list.append(video)
            label_list.append(cl_index)
            index_list.append(s_index)

        data = {"videos": torch.stack(video_list),
                "labels": label_list,
                "indexes": index_list
                }

        return data

    def loader(self):
        return DataLoader(self.dataset,
                          batch_size=self.batch_size,
                          shuffle=self.shuffle,
                          num_workers=self.num_workers,
                          collate_fn=self.collect_fn)

# Mount drive


In [ ]:
import os
import io
from google.colab import drive

# Mount google drive
DRIVE_MOUNT='/content/drive'

drive.mount(DRIVE_MOUNT)


# create folder to write data to
DATA_FOLDER = os.path.join(DRIVE_MOUNT, 'Shared drives', 'CIS680 Final Project', 'data')

Mounted at /content/drive


# Get dataset state files (don't need to run)

In [ ]:
import torchvision.transforms as tf

transform = tf.Compose([tf.Resize(256), tf.RandomCrop(224)])

train_dataset = HMDB51(DATA_FOLDER)
test_dataset = HMDB51(DATA_FOLDER)

train_dataset.init_data(DATA_FOLDER, frames_per_clip=1, transform=transform)
torch.save(train_dataset.state_dict(), DATA_FOLDER + "/train_dataset_1.pt")

test_dataset.init_data(DATA_FOLDER, frames_per_clip=1, train=False, transform=transform)
torch.save(test_dataset.state_dict(), DATA_FOLDER + "/test_dataset_1.pt")

{'brush_hair': 0, 'cartwheel': 1, 'catch': 2, 'chew': 3, 'clap': 4, 'climb': 5, 'climb_stairs': 6, 'dive': 7, 'draw_sword': 8, 'dribble': 9, 'drink': 10, 'eat': 11, 'fall_floor': 12, 'fencing': 13, 'flic_flac': 14, 'golf': 15, 'handstand': 16, 'hit': 17, 'hug': 18, 'jump': 19, 'kick': 20, 'kick_ball': 21, 'kiss': 22, 'laugh': 23, 'pick': 24, 'pour': 25, 'pullup': 26, 'punch': 27, 'push': 28, 'pushup': 29, 'ride_bike': 30, 'ride_horse': 31, 'run': 32, 'shake_hands': 33, 'shoot_ball': 34, 'shoot_bow': 35, 'shoot_gun': 36, 'sit': 37, 'situp': 38, 'smile': 39, 'smoke': 40, 'somersault': 41, 'stand': 42, 'swing_baseball': 43, 'sword': 44, 'sword_exercise': 45, 'talk': 46, 'throw': 47, 'turn': 48, 'walk': 49, 'wave': 50}



{'brush_hair': 0, 'cartwheel': 1, 'catch': 2, 'chew': 3, 'clap': 4, 'climb': 5, 'climb_stairs': 6, 'dive': 7, 'draw_sword': 8, 'dribble': 9, 'drink': 10, 'eat': 11, 'fall_floor': 12, 'fencing': 13, 'flic_flac': 14, 'golf': 15, 'handstand': 16, 'hit': 17, 'hug': 18, 'jump': 19, 'kick': 20, 'kick_ball': 21, 'kiss': 22, 'laugh': 23, 'pick': 24, 'pour': 25, 'pullup': 26, 'punch': 27, 'push': 28, 'pushup': 29, 'ride_bike': 30, 'ride_horse': 31, 'run': 32, 'shake_hands': 33, 'shoot_ball': 34, 'shoot_bow': 35, 'shoot_gun': 36, 'sit': 37, 'situp': 38, 'smile': 39, 'smoke': 40, 'somersault': 41, 'stand': 42, 'swing_baseball': 43, 'sword': 44, 'sword_exercise': 45, 'talk': 46, 'throw': 47, 'turn': 48, 'walk': 49, 'wave': 50}


# Load Dataset and Dataloaders

In [ ]:
import matplotlib.pyplot as plt


train_dataset = HMDB51(DATA_FOLDER)
test_dataset = HMDB51(DATA_FOLDER)

train_dataset.load_state_dict(torch.load(DATA_FOLDER + "/train_dataset_1.pt"))
test_dataset.load_state_dict(torch.load(DATA_FOLDER + "/test_dataset_1.pt"))

batch_size = 128
train_build_loader = BuildDataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
train_loader = train_build_loader.loader()
test_build_loader = BuildDataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = test_build_loader.loader()

print("Length of train_loader: {}".format(len(train_loader)))
print("Length of test_loader: {}".format(len(test_loader)))

Length of train_loader: 3851
Length of test_loader: 1128


# Model

In [ ]:
import torch
from torch.nn import functional as F
from torchvision import transforms
from torch import nn, Tensor
import random
import torchvision
import torchvision.models as models
import torch.optim as optim


class SpatialStream(torch.nn.Module):

    def __init__(self, 
                 device='cuda',
                 num_classes=51,
                 dropout_probability=0.5,
                 train_resnet=True):

        # Initialize the stream layers
        super(SpatialStream, self).__init__()
        self.device = device
        self.num_classes = num_classes

        # Spatial Pretrained Resnet-50
        self.spatial = models.resnet50(pretrained=True)
        for param in self.spatial.parameters():
            param.requires_grad = train_resnet  # False: Freezes the weights of the pre-trained model

        # Add FC layers to Spatial Resnet-50
        self.spatial.fc = nn.Sequential(nn.Linear(2048, 1024),
                                nn.ReLU(),
                                nn.Dropout(p=dropout_probability),
                                nn.Linear(1024, self.num_classes),
                                nn.ReLU(),
                                nn.Dropout(p=dropout_probability),
                                nn.Softmax())

    def forward(self, X):
        return self.spatial(X)

    def compute_loss(self, output, labels):
        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, labels)
        return loss

# Train & Test Functions

In [ ]:
  def train(epoch):
    spatial.train()
    counter = 0
    train_loss = 0
    log_interval = 100
    save_interval = 250

    epoch_loss = []
    log_int_loss = 0

    for iter, data in enumerate(train_loader, 0):

        # load data
        videos = data["videos"]
        labels = torch.tensor(data["labels"])
        indexes = data["indexes"]

        videos = videos.type(torch.FloatTensor)
        videos = videos.to(device)

        optimizer.zero_grad()

        # get spatial output
        output = spatial(videos)
        output = output.to(device)
        labels = labels.to(device)

        # calculate losses
        loss = spatial.compute_loss(output, labels)

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Logging
        log_int_loss += loss.item()
        epoch_loss.append(loss.item())

        if counter == 0:
            print('Epoch: ', epoch, ', Batch: ', iter, ', loss avg over log interval: ', log_int_loss)
            train_loss_list.append(train_loss / (iter + 1) * batch_size)
            train_counter.append((iter + 1) * batch_size + epoch * len(train_loader.dataset))
            log_int_loss = 0
        elif counter % log_interval == log_interval - 1:
            print('Epoch: ', epoch, ', Batch: ', iter, ', loss avg over log interval: ', log_int_loss / log_interval)
            train_loss_list.append(train_loss / (iter + 1) * batch_size)
            train_counter.append((iter + 1) * batch_size + epoch * len(train_loader.dataset))
            log_int_loss = 0

        # Save every save_interval iterations
        if counter % save_interval == save_interval - 1:
            print('saving model')
            save_path = os.path.join(EPOCH_SAVE_PREFIX, 'spatial_epoch' + str(epoch) + '_iter_' + str(counter))
            torch.save({
                'epoch': epoch,
                'train_total_loss_list': train_loss_list,
                'epoch_total_loss_list': epoch_loss_list,
                'test_loss_list': test_loss_list,
                'train_counter': train_counter,
                'accuracy_list': accuracy_list,
                'model_state_dict': spatial.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, save_path)

        counter += 1


    avg_loss = sum(epoch_loss) / len(epoch_loss)
    epoch_loss_list.append(avg_loss)
    print('Epoch: ', epoch, ', avg total loss: ', avg_loss)

def test():
    spatial.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for iter, data in enumerate(test_loader, 0):

            # load data
            videos = data["videos"]
            labels = torch.tensor(data["labels"])
            indexes = data["indexes"]

            videos = videos.type(torch.FloatTensor)
            videos = videos.to(device)
            labels = labels.to(device)

            # get spatial output
            output = spatial(videos)
            output = output.to(device)

            # calculate losses
            loss = spatial.compute_loss(output, labels)

            test_loss += loss.item()

            # calculate number of correct predictions in batch to get accuracy
            correct += sum(torch.argmax(output,1) == labels).item()
            if iter % 100 == 0:
                print ("iter  ", iter)
                print("accuracy so far = ", correct / ((iter + 1) * len(labels)))

    # Log
    test_loss_list.append(test_loss / len(test_loader.dataset))
    accuracy = correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    print('Avg Validation Loss: ', test_loss / len(test_loader.dataset))
    print('Accuracy: ', accuracy)

# Main Training Loop

In [ ]:
EPOCH_SAVE_PREFIX = '/content/drive/Shared drives/CIS680 Final Project/models/spatial/'

# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.random.manual_seed(1)

# model
learning_rate = 0.001
spatial = SpatialStream()
spatial.to(device)
optimizer = optim.SGD(spatial.parameters(), lr=learning_rate, momentum=0.9)

# Epochs
num_epochs = 50

# Logging setup: train
train_loss_list = []
epoch_loss_list = []
train_counter = []

# Logging setup: test
test_loss_list = []
accuracy_list = []
epoch_list = np.arange(num_epochs)

# epoch loop
for epoch in range(num_epochs):

    # Train & Validate
    train(epoch)
    test()

    # Save Model
    save_path = os.path.join(EPOCH_SAVE_PREFIX, 'spatial_epoch' + str(epoch))
    torch.save({
        'epoch': epoch,
        'train_total_loss_list': train_loss_list,
        'epoch_total_loss_list': epoch_loss_list,
        'test_loss_list': test_loss_list,
        'train_counter': train_counter,
        'accuracy_list': accuracy_list,
        'model_state_dict': spatial.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, save_path)

    print("Epoch %d/%d Completed" % (epoch, num_epochs - 1))

/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:117: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch:  0 , Batch:  0 , loss avg over log interval:  3.9320425987243652
Epoch:  0 , Batch:  99 , loss avg over log interval:  3.8924271631240845
Epoch:  0 , Batch:  199 , loss avg over log interval:  3.9314334273338316
saving model
Epoch:  0 , Batch:  299 , loss avg over log interval:  3.930895450115204
Epoch:  0 , Batch:  399 , loss avg over log interval:  3.929605519771576
Epoch:  0 , Batch:  499 , loss avg over log interval:  3.9242276430130003
saving model
Epoch:  0 , Batch:  599 , loss avg over log interval:  3.906380925178528
Epoch:  0 , Batch:  699 , loss avg over log interval:  3.895463914871216
saving model
Epoch:  0 , Batch:  799 , loss avg over log interval:  3.9005392670631407
Epoch:  0 , Batch:  899 , loss avg over log interval:  3.896544728279114
Epoch:  0 , Batch:  999 , loss avg over log interval:  3.890415885448456
saving model
Epoch:  0 , Batch:  1099 , loss avg over log interval:  3.8922301173210143
Epoch:  0 , Batch:  1199 , loss avg over log interval:  3.8916511154

# Resume Training

In [ ]:
EPOCH_SAVE_PREFIX = '/content/drive/Shared drives/CIS680 Final Project/models/spatial/'

# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.random.manual_seed(1)

# model
learning_rate = 0.001
spatial = SpatialStream()
spatial.to(device)
optimizer = optim.SGD(spatial.parameters(), lr=learning_rate, momentum=0.9)

# Epochs
num_epochs = 50
epoch_list = np.arange(num_epochs)


# Load Spatial Network
network_path = '/content/drive/Shared drives/CIS680 Final Project/models/spatial/spatial_epoch8'
checkpoint = torch.load(network_path)
spatial.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']

# Logging setup
train_loss_list = checkpoint['train_total_loss_list']
epoch_loss_list = checkpoint['epoch_total_loss_list']
test_loss_list = checkpoint['test_loss_list']
accuracy_list = checkpoint['accuracy_list']
train_counter = checkpoint['train_counter']


# epoch loop
for epoch in range(last_epoch + 1, num_epochs):

    # Train & Validate
    train(epoch)
    test()

    # Save Model Version
    save_path = os.path.join(EPOCH_SAVE_PREFIX, 'spatial_epoch' + str(epoch))
    torch.save({
        'epoch': epoch,
        'train_total_loss_list': train_loss_list,
        'epoch_total_loss_list': epoch_loss_list,
        'test_loss_list': test_loss_list,
        'train_counter': train_counter,
        'accuracy_list': accuracy_list,
        'model_state_dict': spatial.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, save_path)

    print("Epoch %d/%d Completed" % (epoch, num_epochs - 1))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:117: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch:  9 , Batch:  0 , loss avg over log interval:  3.579054117202759
Epoch:  9 , Batch:  99 , loss avg over log interval:  3.5460293793678286
Epoch:  9 , Batch:  199 , loss avg over log interval:  3.5823871088027954
saving model
Epoch:  9 , Batch:  299 , loss avg over log interval:  3.5742266011238097
Epoch:  9 , Batch:  399 , loss avg over log interval:  3.578859972953796
Epoch:  9 , Batch:  499 , loss avg over log interval:  3.5777490186691283
saving model
Epoch:  9 , Batch:  599 , loss avg over log interval:  3.579861807823181
Epoch:  9 , Batch:  699 , loss avg over log interval:  3.5729906296730043
saving model
Epoch:  9 , Batch:  799 , loss avg over log interval:  3.57385466337204
Epoch:  9 , Batch:  899 , loss avg over log interval:  3.5813414573669435
Epoch:  9 , Batch:  999 , loss avg over log interval:  3.576751308441162
saving model
Epoch:  9 , Batch:  1099 , loss avg over log interval:  3.5745735216140746
Epoch:  9 , Batch:  1199 , loss avg over log interval:  3.5685772466

# Calculate Accuracy of a Saved Model

In [ ]:
# Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model
learning_rate = 0.001
spatial = SpatialStream()
spatial.to(device)
optimizer = optim.SGD(spatial.parameters(), lr=learning_rate, momentum=0.9)

# Load Spatial Network
network_path = '/content/drive/Shared drives/CIS680 Final Project/models/spatial/spatial_epoch0'
checkpoint = torch.load(network_path)
spatial.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']

# Logging setup: train
train_loss_list = checkpoint['train_total_loss_list']
epoch_loss_list = checkpoint['epoch_total_loss_list']
test_loss_list = checkpoint['test_loss_list']
train_counter = checkpoint['train_counter']

test()

/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:117: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
1
2
3
4
5
6
7
8
9
10
iter   10
accuracy so far =  1.0
Avg Validation Loss:  0.0002464993757317368
Accuracy:  0.009755083659542037
